<a href="https://colab.research.google.com/github/hmanoj26/7316/blob/main/kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geemap
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.4 MB/s eta 0:00:00


In [2]:
import os
import ee
import geemap
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import json

import ipywidgets as widgets
from IPython.display import display

In [4]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='remote-sensing-486415')

In [5]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [7]:
image_id = 'LANDSAT/LC09/C02/T1_L2/LC09_034033_20250113'
image = ee.Image(image_id).select('SR_B[1-7]')

# Define the region for statistics. Use the image's own footprint for simplicity.
region_for_stats = image.geometry()

# Calculate mean and standard deviation for dynamic stretching
stats = image.select(['SR_B4', 'SR_B3', 'SR_B2']).reduceRegion(
    reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True),
    geometry=region_for_stats,
    scale=30,
    maxPixels=1e9
).getInfo()

# Define the number of standard deviations for the stretch
std_dev_multiplier = 2

# Extract mean and stdDev values for each band and create dynamic visualization parameters
dynamic_rgb_vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': [
        stats['SR_B4_mean'] - std_dev_multiplier * stats['SR_B4_stdDev'],
        stats['SR_B3_mean'] - std_dev_multiplier * stats['SR_B3_stdDev'],
        stats['SR_B2_mean'] - std_dev_multiplier * stats['SR_B2_stdDev']
    ],
    'max': [
        stats['SR_B4_mean'] + std_dev_multiplier * stats['SR_B4_stdDev'],
        stats['SR_B3_mean'] + std_dev_multiplier * stats['SR_B3_stdDev'],
        stats['SR_B2_mean'] + std_dev_multiplier * stats['SR_B2_stdDev']
    ]
}

Map.centerObject(image.geometry(), 8)
Map.addLayer(image, dynamic_rgb_vis_params, "Landsat-9 - LC09_034033_20250113 (Dynamic Stretch)")

In [9]:
props = geemap.image_props(image)
props.getInfo()

{'ALGORITHM_SOURCE_SURFACE_REFLECTANCE': 'LaSRC_1.6.0',
 'ALGORITHM_SOURCE_SURFACE_TEMPERATURE': 'st_1.5.0',
 'CLOUD_COVER': 6.66,
 'CLOUD_COVER_LAND': 6.66,
 'COLLECTION_CATEGORY': 'T1',
 'COLLECTION_NUMBER': 2,
 'DATA_SOURCE_AIR_TEMPERATURE': 'VIIRS',
 'DATA_SOURCE_ELEVATION': 'GLS2000',
 'DATA_SOURCE_OZONE': 'VIIRS',
 'DATA_SOURCE_PRESSURE': 'Calculated',
 'DATA_SOURCE_REANALYSIS': 'GEOS-5 IT',
 'DATA_SOURCE_WATER_VAPOR': 'VIIRS',
 'DATE_ACQUIRED': '2025-01-13',
 'DATE_PRODUCT_GENERATED': 1736865104000,
 'DATUM': 'WGS84',
 'EARTH_SUN_DISTANCE': 0.9835492,
 'ELLIPSOID': 'WGS84',
 'GEOMETRIC_RMSE_MODEL': 7.968,
 'GEOMETRIC_RMSE_MODEL_X': 5.134,
 'GEOMETRIC_RMSE_MODEL_Y': 6.094,
 'GEOMETRIC_RMSE_VERIFY': 4.511,
 'GRID_CELL_SIZE_REFLECTIVE': 30,
 'GRID_CELL_SIZE_THERMAL': 30,
 'GROUND_CONTROL_POINTS_MODEL': 524,
 'GROUND_CONTROL_POINTS_VERIFY': 129,
 'GROUND_CONTROL_POINTS_VERSION': 5,
 'IMAGE_DATE': '2025-01-13',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS': 9,
 'L1_DATE_PRODUCT_GENE

In [10]:
props.get("IMAGE_DATE").getInfo()

'2025-01-13'

In [11]:
props.get("CLOUD_COVER").getInfo()

6.66

In [12]:
# Make the training dataset.
training = image.sample(
    **{
        #     'region': region,
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(training, {}, "training", False)

In [15]:
# Instantiate the clusterer and train it.
n_clusters = 6
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [16]:
# Cluster the input using the trained clusterer.
result = image.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, "clusters")

In [17]:
from ipyleaflet import WidgetControl

# Remove existing geemap legend controls before adding a new one
for control in list(Map.controls): # Iterate over a copy to modify in-place
    if isinstance(control, WidgetControl) and getattr(control, 'name', None) == 'Legend':
        Map.remove_control(control)

legend_keys = ["New Label 1", "New Label 2", "New Label 3", "New Label 4", "New Label 5", "New Label 6"] # Update these with your desired labels
legend_colors = ["#8DD3C7", "#FFFFB3", "#BEBADA", "#FB8072", "#80B1D3", "#8DB2D9"]

# Reclassify the map
result = result.remap([0, 1, 2, 3, 4, 5], [1, 2, 3, 4, 5, 6])

Map.addLayer(
    result, {"min": 1, "max": 6, "palette": legend_colors}, "Labelled clusters"
)
Map.add_legend(
    legend_keys=legend_keys, legend_colors=legend_colors, position="bottomright"
)